In [2]:
from tensorflow.keras.models import load_model
model = load_model('./models_log/11_spcup_best_weight.h5')
model = load_model('C:/Users/GJ/Downloads/best_model/49_spcup_best.h5')

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_21 (Batc (None, 64, 32, 1)         4         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 64, 32, 16)        416       
_________________________________________________________________
batch_normalization_22 (Batc (None, 64, 32, 16)        64        
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 64, 32, 32)        12832     
_________________________________________________________________
batch_normalization_23 (Batc (None, 64, 32, 32)        128       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 64, 32, 64)        51264     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 12, 6, 64)        

In [4]:
%matplotlib inline
from tensorflow import keras
import matplotlib.pyplot as plt
import os
from scipy.io import wavfile
from collections import defaultdict, Counter
from scipy import signal
import numpy as np
import librosa
import random as rn
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
#from keras.engine import Model
from tensorflow.keras.layers import Dense, TimeDistributed, Dropout, Bidirectional, GRU, BatchNormalization, Activation, LeakyReLU, LSTM, Flatten, RepeatVector, Permute, Multiply, Conv2D, MaxPooling2D
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# data_dir = '/content/drive/MyDrive/Dataset/audiofile_shorted500/'
# volume_path = 'C:/Users/GJ/Desktop/연구실/2022SPCUP/WAV_Ordered train set'
volume_path = 'C:/Users/GJ/Desktop/연구실/2022SPCUP/spcup_2022_training_part1'


FilePathList =[]

for dirName, subdirList, fileList in os.walk(volume_path):
    for filename in fileList:
        if '.csv' not in filename:
#             print(filename)
            FilePathList.append(volume_path+'/'+filename)
        else:
            pd_label = pd.read_csv(volume_path+'/'+filename)

labels = np.array(pd_label['algorithm'])


sampling_rate = 16000 
n_mels = 64
hop_length = 512
n_fft = 2048
rs = 10

##train set
print('raw train_set_num :',len(labels))
X_train_path, X_test_path, y_train_raw, y_test_raw = train_test_split(np.array(FilePathList), 
                                                                      labels, test_size=0.2, 
                                                                      stratify = labels, random_state=rs)
y_labels_aug = []
X_mel_aug = []
file_split_num_train = []
for i, filepath in enumerate(X_train_path):
    # fig, ax = plt.subplots()
    y, sr = librosa.load(filepath, mono=True, sr = sampling_rate)
    index_f = 0
    for j in range(y.size//sampling_rate):
        X_aug_seg = y[index_f:index_f+sampling_rate]


        S = librosa.feature.melspectrogram(y=X_aug_seg, sr=sampling_rate, n_mels=n_mels, fmax=8000,
                                           hop_length = hop_length,n_fft = n_fft)
        S_dB = librosa.power_to_db(S, ref=np.max)
        X_mel_aug.append(S_dB)

        y_labels_aug.append(y_train_raw[i])

        index_f = index_f+sampling_rate
    file_split_num_train.append(j+1)
        #         X_aug_seg = np.expand_dims(X_aug_seg, axis=0)
        #         X_aug.append(X_aug_seg)
# X_aug = np.concatenate(X_aug,axis = 0)
X_train1 = np.array(X_mel_aug)
y_train1 = np.array(y_labels_aug)
print('train set.shape : ',X_train1.shape)
print('y_train.shape',y_train1.shape )
print('len(file_split_num_train)',len(file_split_num_train))

X_train = X_train1.reshape(X_train1.shape[0],X_train1.shape[1],X_train1.shape[2],1)
print('X_train.shape : ',X_train.shape)

y_train = to_categorical(y_train1) 
# print(y_test.shape)
print('y_train.shape',y_train.shape)



raw train_set_num : 5000
train set.shape :  (25824, 64, 32)
y_train.shape (25824,)
len(file_split_num_train) 4000
X_train.shape :  (25824, 64, 32, 1)
y_train.shape (25824, 5)


In [5]:
print(X_train[0])

[[[-73.86101 ]
  [-59.077103]
  [-37.841267]
  ...
  [-48.186077]
  [-36.118088]
  [-33.09801 ]]

 [[-76.81702 ]
  [-62.228462]
  [-43.45993 ]
  ...
  [-36.269817]
  [-30.334215]
  [-30.211031]]

 [[-80.      ]
  [-66.469246]
  [-49.74462 ]
  ...
  [-43.3544  ]
  [-38.120316]
  [-37.507824]]

 ...

 [[-80.      ]
  [-80.      ]
  [-64.54021 ]
  ...
  [-63.23665 ]
  [-61.350937]
  [-60.38127 ]]

 [[-80.      ]
  [-80.      ]
  [-71.56448 ]
  ...
  [-72.57238 ]
  [-67.71051 ]
  [-62.462082]]

 [[-80.      ]
  [-80.      ]
  [-73.916664]
  ...
  [-80.      ]
  [-73.4952  ]
  [-63.760674]]]


In [6]:
print(y_train_raw[:10])

[4 4 3 4 3 3 2 4 4 3]


In [8]:
train_loss, train_acc = model.evaluate(X_train,  y_train, verbose=2)

print('테스트 정확도:', train_acc)

prediction_train = model.predict(X_train)

last_index = 0
for i in file_split_num_train[:10]:
    print(np.round(prediction_train[last_index:i+last_index],3))
    print('\n')
    print(np.round(y_train[last_index:i+last_index],3))
    print('\n')
    print('-------------------------')
    
    
    last_index += i

807/807 - 4s - loss: 0.0359 - accuracy: 0.9890
테스트 정확도: 0.9890412092208862
[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


-------------------------
[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


-------------------------
[[0.    0.    0.    0.999 0.   ]
 [0.    0.    0.    1.    0.   ]
 [0.    0.    0.    1.    0.   ]
 [0.    0.    0.    1.    0.   ]
 [0.    0.    0.    1.    0.   ]
 [0.    0.    0.    1.    0.   ]
 [0.    0.    0.    1.    0.   ]
 [0.    0.    0.    1.    0.   ]]


[[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]


-------------------------
[[0. 0. 0. 0. 1.]
 [0. 0.

In [9]:
train_loss, train_acc = model.evaluate(X_test,  y_test, verbose=2)

print('테스트 정확도:', train_acc)

NameError: name 'X_test' is not defined

In [10]:
#################Unseen data Test

# data_dir = '/content/drive/MyDrive/Dataset/audiofile_shorted500/'
# volume_path = 'C:/Users/GJ/Desktop/연구실/2022SPCUP/WAV_Ordered train set'
volume_path = './spcup_2022_unseen'


UnseenFilePathList =[]

for dirName, subdirList, fileList in os.walk(volume_path):
    for filename in fileList:
        if '.csv' not in filename:
#             print(filename)
            UnseenFilePathList.append(volume_path+'/'+filename)
        else:
            pd_label = pd.read_csv(volume_path+'/'+filename)

unseen_labels = np.array(pd_label['algorithm'])

##test set

y_labels_aug = []
X_mel_aug = []
file_split_num_test = []
for i, filepath in enumerate(UnseenFilePathList):
    # fig, ax = plt.subplots()
    y, sr = librosa.load(filepath, mono=True, sr = sampling_rate)
    index_f = 0
    for j in range(y.size//sampling_rate):
        X_aug_seg = y[index_f:index_f+sampling_rate]


        S = librosa.feature.melspectrogram(y=X_aug_seg, sr=sampling_rate, n_mels=n_mels,
                                           fmax=8000, hop_length = hop_length, n_fft = n_fft)
        S_dB = librosa.power_to_db(S, ref=np.max)
        X_mel_aug.append(S_dB)

        y_labels_aug.append(unseen_labels[i])

        index_f = index_f+sampling_rate
    file_split_num_test.append(j+1)

X_test1 = np.array(X_mel_aug)
y_test1 = np.array(y_labels_aug)
print('splited_test set.shape : ',X_test1.shape)
print('y_test.shape : ', y_test1.shape)
print(len(file_split_num_test))


X_test = X_test1.reshape(X_test1.shape[0],X_test1.shape[1],X_test1.shape[2],1)
print('X_train.shape : ',X_train.shape)
print('X_test.shape : ',X_test.shape)

y_test = to_categorical(y_test1) 
# print(y_test.shape)
print('y_train.shape',y_train.shape)
print('y_test.shape : ',y_test.shape)



splited_test set.shape :  (6278, 64, 32)
y_test.shape :  (6278,)
1000
X_train.shape :  (25824, 64, 32, 1)
X_test.shape :  (6278, 64, 32, 1)
y_train.shape (25824, 5)
y_test.shape :  (6278, 6)


In [41]:
y_test = y_test[:][:,0:5]
print(y_test.shape)

(6278, 5)


In [45]:
y_test = y_test[:][:,0:5]
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('테스트 정확도:', test_acc)

prediction_test = model.predict(X_test)

last_index = 0

for i in file_split_num_test:
    print('\n')
    print(np.round(prediction_test[last_index:i+last_index],2))
    print(np.round(y_test[last_index:i+last_index],2))
    print('\n')
    print('--------------------------')
    last_index += i

197/197 - 1s - loss: 0.0000e+00 - accuracy: 0.0000e+00
테스트 정확도: 0.0


[[0.   0.84 0.16 0.   0.  ]
 [0.   1.   0.   0.   0.  ]
 [0.   0.   1.   0.   0.  ]
 [0.   0.99 0.01 0.   0.  ]
 [0.   1.   0.   0.   0.  ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--------------------------


[[0.   1.   0.   0.   0.  ]
 [0.   0.01 0.99 0.   0.  ]
 [0.   0.98 0.02 0.   0.  ]
 [0.   0.94 0.06 0.   0.  ]
 [0.   1.   0.   0.   0.  ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--------------------------


[[0.   0.61 0.39 0.   0.  ]
 [0.   0.01 0.99 0.   0.  ]
 [0.   0.83 0.17 0.   0.  ]
 [0.   0.94 0.   0.06 0.  ]
 [0.   0.66 0.33 0.01 0.  ]
 [0.   0.75 0.25 0.   0.  ]
 [0.   0.   1.   0.   0.  ]
 [0.   0.69 0.3  0.   0.  ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--------------------------


[[0

 [0.   0.   1.   0.   0.  ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--------------------------


[[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--------------------------


[[0.   0.   1.   0.   0.  ]
 [0.   0.16 0.84 0.   0.  ]
 [0.   0.82 0.18 0.   0.  ]
 [0.   1.   0.   0.   0.  ]
 [0.   0.14 0.86 0.   0.  ]
 [0.   0.4  0.6  0.   0.  ]
 [0.   0.93 0.07 0.   0.  ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--------------------------


[[0.   0.   1.   0.   0.  ]
 [0.   0.04 0.96 0.   0.  ]
 [0.   0.99 0.01 0.   0.  ]
 [0.   0.   0.03 0.97 0.  ]
 [0.   0.03 0.09 0.88 0.  ]
 [0.   1.   0.   0.   0.  ]
 [0.   0.   1.   0.   0.  ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0

 [0. 0. 0. 0. 0.]]


--------------------------


[[0.   0.   1.   0.   0.  ]
 [0.   0.12 0.88 0.   0.  ]
 [0.   0.07 0.93 0.   0.  ]
 [0.   1.   0.   0.   0.  ]
 [0.   0.92 0.08 0.   0.  ]
 [0.   1.   0.   0.   0.  ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--------------------------


[[0.   0.   1.   0.   0.  ]
 [0.   0.01 0.99 0.   0.  ]
 [0.   1.   0.   0.   0.  ]
 [0.   0.06 0.94 0.   0.  ]
 [0.   0.84 0.16 0.   0.  ]
 [0.   1.   0.   0.   0.  ]
 [0.   1.   0.   0.   0.  ]
 [0.   0.   1.   0.   0.  ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--------------------------


[[0.   0.99 0.01 0.   0.  ]
 [0.   0.36 0.64 0.   0.  ]
 [0.   0.98 0.02 0.   0.  ]
 [0.   0.99 0.01 0.   0.  ]
 [0.   0.58 0.28 0.14 0.  ]
 [0.   0.   1.   0.   0.  ]
 [0.   0.   1.   0.   0.  ]
 [0.   1.   0.   0.   0.  ]]
[[0. 0. 0. 0. 0


[[0.   0.11 0.89 0.   0.  ]
 [0.   0.05 0.   0.95 0.  ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--------------------------


[[0.   0.   1.   0.   0.  ]
 [0.   0.   1.   0.   0.  ]
 [0.   0.   1.   0.   0.  ]
 [0.   0.17 0.83 0.   0.  ]
 [0.   0.   1.   0.   0.  ]
 [0.   0.06 0.94 0.   0.  ]
 [0.   0.   1.   0.   0.  ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--------------------------


[[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--------------------------


[[0.   0.16 0.84 0.   0.  ]
 [0.   1.   0.   0.   0.  ]
 [0.   0.   1.   0.   0.  ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--------------------

 [0.   0.   1.   0.   0.  ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--------------------------


[[0. 0. 1. 0. 0.]]
[[0. 0. 0. 0. 0.]]


--------------------------


[[0. 0. 1. 0. 0.]]
[[0. 0. 0. 0. 0.]]


--------------------------


[[0.   0.02 0.98 0.   0.  ]
 [0.   1.   0.   0.   0.  ]
 [0.   0.   1.   0.   0.  ]
 [0.   0.82 0.04 0.14 0.  ]
 [0.   1.   0.   0.   0.  ]
 [0.   0.05 0.95 0.   0.  ]
 [0.   0.99 0.01 0.   0.  ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--------------------------


[[0.   0.74 0.26 0.   0.  ]
 [0.   0.   1.   0.   0.  ]
 [0.   0.   1.   0.   0.  ]
 [0.   0.   1.   0.   0.  ]
 [0.   0.   1.   0.   0.  ]
 [0.   0.05 0.95 0.   0.  ]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


----